# ACDI assessment

## Task 1: Climate Data Analysis with Moving Window Distribution Parameters
**Estimated Time:** ~4 hours

### Problem Description
You are given a large 3-D climate dataset with (time, latitude, longitude) dimensions representing daily temperature data over ~30 years.

For each grid cell (lat, lon), compute distribution parameters over a moving temporal window of 90 days and return a new dataset of dimensions (time, latitude, longitude) containing one of the fitted parameters.

### Dataset
This is your data array (~5 GB of data):

**Python:**
```python
import numpy as np
X = np.random.normal(loc=20, scale=4, size=(10950, 180, 365))
```

**R:**
```r
X <- array(
  rnorm(10950 * 180 * 365, mean = 20, sd = 4),
  dim = c(10950, 180, 365)
)
```

### Requirements
Design your solution accordingly using parallelization.

## Solution

### Packages

`bottleneck` is installed to speed up xarray's rolling calculations — it provides fast C implementations of moving window functions and is used by xarray automatically if installed. Dask provides chunked, parallelisable arrays that allow computation to be distributed across multiple threads.

In [9]:
import numpy as np
import pandas as pd
import xarray as xr
import dask.array as da
from dask.diagnostics import ProgressBar

### Data

The dataset is created as specified in the task using `np.random.normal()`. Ideally, data would be generated directly with `da.random.normal()`, which creates a dask array lazily without allocating the full array in the main process — avoiding the memory overhead of `da.from_array()`. We leave this commented out to remain true to the task specification.

The numpy array is then wrapped in a dask array chunked along lat and lon (`chunks=(-1, 30, 30)`). Note that chunking does not reduce memory here — `da.from_array()` references `X` in-place, so the full 5.37 GB stays in RAM regardless. Memory savings from chunking only apply when loading lazily from disk. Here, chunking solely provides parallelism:

In [10]:
X = np.random.normal(loc=20, scale=4, size=(10950, 180, 365))

# alternatively: Xda = da.random.normal(loc=20, scale=4, size=(10950, 180, 365))
Xda = da.from_array(X, chunks=(-1, 30, 30))
Xda

dask.array<array, shape=(10950, 180, 365), dtype=float64, chunksize=(10950, 30, 30), chunktype=numpy.ndarray>

In [11]:

#create coordinates
end_date = pd.Timestamp.today().normalize()
time_coords = pd.date_range(end=end_date, periods=10950, freq='D')
lat_coords = np.linspace(-90, 90, 180)
lon_coords = np.linspace(-180, 180, 365)

#create xarray
ds = xr.Dataset(
    {"temperature": (["time", "lat", "lon"], Xda)},
    coords={
        "time": time_coords,
        "lat": lat_coords,
        "lon": lon_coords,
    },
)
ds


<xarray.Dataset> Size: 6GB
Dimensions:      (time: 10950, lat: 180, lon: 365)
Coordinates:
  * time         (time) datetime64[us] 88kB 1996-02-27 1996-02-28 ... 2026-02-18
  * lat          (lat) float64 1kB -90.0 -88.99 -87.99 ... 87.99 88.99 90.0
  * lon          (lon) float64 3kB -180.0 -179.0 -178.0 ... 178.0 179.0 180.0
Data variables:
    temperature  (time, lat, lon) float64 6GB dask.array<chunksize=(10950, 30, 30), meta=np.ndarray>

### Rolling Window Statistics

A 90-day centred rolling window is computed for each grid cell. `min_periods=window` ensures that any timestep with fewer than 90 valid values in its window is set to NaN — this affects the first and last 45 timesteps at the edges of the time series.

Mean, standard deviation, min, and max are computed directly via xarray's rolling methods. Quantiles require first constructing an explicit window dimension via `roller.construct()`, which creates a view of shape `(time, lat, lon, 90)`, before applying the quantile reduction along that dimension.

In [12]:

window = 90
roller = ds.rolling(time=window, center=True, min_periods=window)

roll_mean = roller.mean()["temperature"]
roll_std = roller.std()["temperature"]
roll_min = roller.min()["temperature"]
roll_max = roller.max()["temperature"]

# quantiles: https://stackoverflow.com/questions/54607236/rolling-quantile-with-xarray
roll_constructed = roller.construct("window_dim")["temperature"]
roll_p05 = roll_constructed.quantile(0.05, dim="window_dim", skipna=False).drop_vars("quantile")
roll_p95 = roll_constructed.quantile(0.95, dim="window_dim", skipna=False).drop_vars("quantile")

ds_stats = xr.Dataset({
    "mean": roll_mean,
    "std": roll_std,
    "min": roll_min,
    "max": roll_max,
    "p05": roll_p05,
    "p95": roll_p95,
})
ds_stats


<xarray.Dataset> Size: 35GB
Dimensions:  (time: 10950, lat: 180, lon: 365)
Coordinates:
  * time     (time) datetime64[us] 88kB 1996-02-27 1996-02-28 ... 2026-02-18
  * lat      (lat) float64 1kB -90.0 -88.99 -87.99 -86.98 ... 87.99 88.99 90.0
  * lon      (lon) float64 3kB -180.0 -179.0 -178.0 -177.0 ... 178.0 179.0 180.0
Data variables:
    mean     (time, lat, lon) float64 6GB dask.array<chunksize=(10863, 30, 30), meta=np.ndarray>
    std      (time, lat, lon) float64 6GB dask.array<chunksize=(10863, 30, 30), meta=np.ndarray>
    min      (time, lat, lon) float64 6GB dask.array<chunksize=(10863, 30, 30), meta=np.ndarray>
    max      (time, lat, lon) float64 6GB dask.array<chunksize=(10863, 30, 30), meta=np.ndarray>
    p05      (time, lat, lon) float64 6GB dask.array<chunksize=(10949, 3, 3), meta=np.ndarray>
    p95      (time, lat, lon) float64 6GB dask.array<chunksize=(10949, 3, 3), meta=np.ndarray>

In [13]:
#force computation and return numpy-backed xarray
with ProgressBar():
    mean = ds_stats['mean'].compute()

[########################################] | 100% Completed | 10.46 ss


In [14]:
mean

<xarray.DataArray 'mean' (time: 10950, lat: 180, lon: 365)> Size: 6GB
array([[[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
...
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]]], shape=(10950, 180, 365))
Coordinates:
  * time     (time) datetime64[us] 88kB 1996-02-27 1996-02-28 ... 2026-02-18
  * lat      (lat) float64 1kB -90.0 -88.99 -87.99 -86.98 ... 87.99 88.99 90.0
  * lon      (lon) float64 3kB -180.0 -179.0 -178.0 -177.0 ... 178.0 179.0 180.0

In [15]:
#verify that mean is correct. ignore nan
np.nanmean(mean.values)

np.float64(20.000082704234956)